In [1]:
#read lines of vent from day5.txt
with open('day5.txt') as f:
    lines = f.readlines()
    lines = [line.strip() for line in lines]

# lines = """0,9 -> 5,9
# 8,0 -> 0,8
# 9,4 -> 3,4
# 2,2 -> 2,1
# 7,0 -> 7,4
# 6,4 -> 2,0
# 0,9 -> 2,9
# 3,4 -> 1,4
# 0,0 -> 8,8
# 5,5 -> 8,2""".split('\n')
# lines = [line.strip() for line in lines]



In [2]:
# the submarine helpfully produces a list of nearby lines of vents (your puzzle input) for you to review. For example:

# 0,9 -> 5,9
# 8,0 -> 0,8
# 9,4 -> 3,4
# 2,2 -> 2,1
# 7,0 -> 7,4
# 6,4 -> 2,0
# 0,9 -> 2,9
# 3,4 -> 1,4
# 0,0 -> 8,8
# 5,5 -> 8,2

# Each line of vents is given as a line segment in the format x1,y1 -> x2,y2 where x1,y1 are the coordinates of one end the line segment and x2,y2 are the coordinates of the other end. These line segments include the points at both ends. In other words:

# An entry like 1,1 -> 1,3 covers points 1,1, 1,2, and 1,3.
# An entry like 9,7 -> 7,7 covers points 9,7, 8,7, and 7,7.

# For each line, get convers points and add them to a set.
def get_convers_points(line):
    x1, y1 = line.split('->')[0].split(',')
    x2, y2 = line.split('->')[1].split(',')
    convers_points = set()
    if int(x1) == int(x2):
        for y in range(min(int(y1), int(y2)), max(int(y1), int(y2)) + 1):
            convers_points.add((int(x1), y))
    if int(y1) == int(y2):
        for x in range(min(int(x1), int(x2)), max(int(x1), int(x2)) + 1):
            convers_points.add((x, int(y1)))
    return convers_points

convers_points = set()
for line in lines:
    cps = get_convers_points(line)
    convers_points = convers_points.union(cps)

# max x of convers points
maxx = max(convers_points, key=lambda x: x[0])[0]
#get max y of convers points
maxy = max(convers_points, key=lambda x: x[1])[1]
print(maxx, maxy)
print(len(convers_points))



986 989
102372


In [8]:
# For now, only consider horizontal and vertical lines: lines where either x1 = x2 or y1 = y2.

# So, the horizontal and vertical lines from the above list would produce the following diagram:

# .......1..
# ..1....1..
# ..1....1..
# .......1..
# .112111211
# ..........
# ..........
# ..........
# ..........
# 222111....
# In this diagram, the top left corner is 0,0 and the bottom right corner is 9,9. Each position is shown as the number of lines which cover that point or . if no line covers that point. The top-left pair of 1s, for example, comes from 2,2 -> 2,1; the very bottom row is formed by the overlapping lines 0,9 -> 5,9 and 0,9 -> 2,9.

# To avoid the most dangerous areas, you need to determine the number of points where at least two lines overlap. In the above example, this is anywhere in the diagram with a 2 or larger - a total of 5 points.

# Consider only horizontal and vertical lines. At how many points do at least two lines overlap?

{(7, 7), (8, 7), (9, 7)}


In [3]:
def get_count_map(lines):
# Consider only horizontal and vertical lines.
# At how many points do at least two lines overlap?
    count_map = {}
    for line in lines:
        x1, y1 = line.split('->')[0].split(',')
        x2, y2 = line.split('->')[1].split(',')
        if int(x1) == int(x2):
            for y in range(min(int(y1), int(y2)), max(int(y1), int(y2)) + 1):
                count_map[(int(x1), y)] = count_map.get((int(x1), y), 0) + 1
        if int(y1) == int(y2):
            for x in range(min(int(x1), int(x2)), max(int(x1), int(x2)) + 1):
                count_map[(x, int(y1))] = count_map.get((x, int(y1)), 0) + 1
    return count_map

count_map = get_count_map(lines)


def print_count_map(count_map):
    for y in range(maxy+1):
        for x in range(maxx+1):
            print(count_map.get((x, y), '.'), end='')
        print()
# the number of points where at least two lines overlap. In the above example, this is anywhere in the diagram with a 2 or larger - a total of 5 points.
def get_number_of_points_with_two_or_more_lines(count_map):
    number_of_points_with_two_or_more_lines = 0
    for key, value in count_map.items():
        if value >= 2:
            number_of_points_with_two_or_more_lines += 1
    return number_of_points_with_two_or_more_lines

print(get_number_of_points_with_two_or_more_lines(count_map))


5698


In [4]:
#Unfortunately, considering only horizontal and vertical lines doesn't give you the full picture; you need to also consider diagonal lines.

# Because of the limits of the hydrothermal vent mapping system, the lines in your list will only ever be horizontal, vertical, or a diagonal line at exactly 45 degrees. In other words:

# An entry like 1,1 -> 3,3 covers points 1,1, 2,2, and 3,3.
# An entry like 9,7 -> 7,9 covers points 9,7, 8,8, and 7,9.
def get_diagonal_lines(lines):
    diagonal_lines = []
    for line in lines:
        x1, y1 = line.split('->')[0].split(',')
        x2, y2 = line.split('->')[1].split(',')
        if abs(int(x1) - int(x2)) == abs(int(y1) - int(y2)) and int(x1) != int(x2) and int(y1) != int(y2):
            diagonal_lines.append(line)
    return diagonal_lines

diagonal_lines = get_diagonal_lines(lines)
def add_diagonal_lines(count_map, diagonal_lines):
    for line in diagonal_lines:
        x1, y1 = line.split('->')[0].split(',')
        x2, y2 = line.split('->')[1].split(',')
        # a 45 degree line is a diagonal line
        # get a direction list from x1 to x2
        listx = range(int(x1), int(x2) + 1) if int(x1) < int(x2) else range(int(x1), int(x2) - 1, -1)
        # get a direction list from y1 to y2
        listy = range(int(y1), int(y2) + 1) if int(y1) < int(y2) else range(int(y1), int(y2) - 1, -1)        
        for x,y in zip(listx, listy):
            count_map[(x, y)] = count_map.get((x, y), 0) + 1
    return count_map

count_map = add_diagonal_lines(count_map, diagonal_lines)

print(get_number_of_points_with_two_or_more_lines(count_map))


15463
